# Lab 2. Inventory Management

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent contains instructions on how to manage and maintain inventory for different product lines, where retail managers can ask the agent to return these information from an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/).

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if the answer to a question is not present on the Knowledge Base, store managers can ask the agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/solar_panel_agent.png)

## Setup

Make sure that your boto3 version is the latest

If not, return to [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run the Setup block again.

In [ ]:
!pip freeze | grep boto3

Get your workshop ID / resource suffix. If not found, return to [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run the Setup block again.

In [ ]:
import os

def get_workshop_id():
    workshop_id_file = '../.workshop_id'
    if os.path.exists(workshop_id_file):
        with open(workshop_id_file, 'r') as f:
            return f.read().strip()
    else:
        return None
    
workshop_id = get_workshop_id()
resource_suffix = f"{workshop_id}"

if workshop_id is None:
    print("No workshop ID found. Please run the Setup script in notebook 1.")
else:
    print("Your resource suffix is", resource_suffix)

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create out second agent.

In [ ]:
import boto3
import os

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [ ]:
inventory_agent_name = f"inventory-mgmt-{resource_suffix}"

inventory_lambda_name = f"fn-inventory-mgmt-{resource_suffix}"

inventory_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{inventory_agent_name}'

dynamodb_table = f"{inventory_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'inv-kb-{resource_suffix}'

knowledge_base_description = "KB containing inventory management guidelines for stocking and maintenance"
bucket_name = f'inventory-mgmt-kb-{account_id}-{resource_suffix}'

### Importing helper functions

In the following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

In `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

In `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

In this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle inventory management

**This creation process takes several minutes.**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [ ]:
def invoke_bedrock_generate_inventory_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close()

Generating one file with instructions on inventory stocking guidelines.

In [ ]:
text_generation_inventory_instructions = '''
    You will be act as an expert in retail inventory management.
    You will generate guidelines on how to stock inventory for different product lines in a retail setting.
    You know the following fictional product lines: Premium Series, Standard Collection, and Value Basics.
    For each product line, provide some general description and its features. 
    Next provide a numbered list describing how to properly stock each product line in a retail store. 
    Include information about how to ensure compliance with company inventory standards.

    Answer only with the instructions and product line descriptions.
    Avoid answer with affirmations like: "OK, I can generate it,",
    "As an expert on inventory management, I ", or "Yes, please find following example."
    Be direct and only reply the instructions and descriptions.
'''

inventory_file_name = 'inventory-stocking-guidelines.txt'

response_message = invoke_bedrock_generate_inventory_files(
    text_generation_inventory_instructions
)
description_and_instructions = response_message['content'][0]['text']
print(f"Generated data:\n{description_and_instructions}")
write_file(
    '{}/{}'.format(path, inventory_file_name),
    description_and_instructions
)

### Generating data prompt
Generating another file with instructions on inventory maintenance procedures.

In [ ]:
text_generation_inventory_maintenance = f'''
    You will be act as an expert in retail inventory management.
    You know the following fictional product lines: Premium Series, Standard Collection, and Value Basics. 
    Here are some descriptions of the different product lines and how to stock them:
    <description_and_instructions>
    {description_and_instructions}
    </description_and_instructions>
    Generate step-by-step instructions on how to properly maintain inventory for each of
    these product lines in a retail store setting. Include information about
    how to ensure consistent compliance with company inventory standards.
    Format your response as a numbered list for each product line.
'''

inventory_maintenance_file_name = 'inventory-maintenance.txt'

response_message = invoke_bedrock_generate_inventory_files(
    text_generation_inventory_maintenance
)
print(f"Generated data:\n{response_message['content'][0]['text']}")

write_file(
    '{}/{}'.format(path, inventory_maintenance_file_name),
    response_message['content'][0]['text']
)

### Uploading data to S3
Uploading generated files into S3 Bucket.

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Creating Agent

Create the Inventory Management agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
You are an Inventory Management Assistant that helps store managers with inventory stocking and maintenance guidance.

Your capabilities include:
1. Providing stocking instructions
2. Providing inventory maintenance procedures
3. Troubleshooting common inventory issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking managers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user
```
We will also connect a knowledge base with the information about inventory management

```
Access the knowledge base when users ask about inventory stocking and maintenance procedures
```
And we will make the following tool available to the agent:
- `open_ticket`: to open new support tickets
- `get_ticket_status`: to get the current status of an existing ticket

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when users ask about inventory stocking and maintenance procedures'
}

In [ ]:
agent_instruction = """You are an Inventory Management Assistant that helps store managers with inventory stocking and maintenance guidance.

Your capabilities include:
1. Providing stocking instructions
2. Offering inventory maintenance procedures
3. Troubleshooting common inventory issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking managers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are an inventory management helper bot. 
    You can retrieve information on how to stock and maintain inventory for different product lines"""

inventory_agent = agents.create_agent(
    inventory_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
inventory_agent

In [ ]:
agents.associate_kb_with_agent(
    inventory_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

### Creating Action Group

On this session, we're going create an action group to handle support tickets and associate it with our agent. To do so, we will first create a Lambda function code to fulfill the execution of the agent's actions Next we will define the actions available actions that an agent can take using function details. Similar to the previous agent, you can also define the actions available using OpenAPI Schema.

#### Creating Lambda function
First let's create the lambda function

In [ ]:
%%writefile inventory.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(store_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': store_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return f"Thanks for contacting inventory support, Store #{store_id}! Your support case was generated with ID: {ticket_id}"

def get_ticket_status(store_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         store_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    store_id = get_named_parameter(event, "store_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(store_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(store_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

### Defining available actions
Now it's time to define the actions that can be taken by the agent

In [ ]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related to inventory management""",
        "parameters": {
            "store_id": {
                "description": "Unique store identifier",
                "required": True,
                "type": "string"
            },
            "msg": {
                "description": "The reason why store manager is opening a ticket",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
            "store_id": {
                "description": "Unique store identifier",
                "required": True,
                "type": "string"
            },
            "ticket_id": {
                "description": "Unique ticket identifier",
                "required": False,
                "type": "string"
            }
        }
    }
]

#### Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [ ]:
resp = agents.add_action_group_with_lambda(
    agent_name=inventory_agent_name,
    lambda_function_name=inventory_lambda_name,
    source_code_file="inventory.py",
    agent_functions=functions_def,
    agent_action_group_name="inventory_actions",
    agent_action_group_description="Function to open an inventory support ticket for a store or get status from an opened ticket",
    dynamo_args=dynamoDB_args
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing maintenance question
First let's ask a question related to maintaining inventory

In [ ]:
%%time
response = agents.invoke(
    "how can I check if my Premium Series inventory is compliant with company standards?", 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(response)

### Testing stocking question
Next let's ask a question related to stocking inventory

In [ ]:
%%time
response = agents.invoke(
    "how should I stock Value Basics products in my store?", 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(response)

### Testing personalized support
Let's now create a support ticket. To do so we will pass our store id to trigger the support ticket creation process

In [ ]:
%%time
ticket = agents.invoke(
    "I need help setting up the Premium Series display in my store. My store id is 1", 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(ticket)

### Testing getting support ticket details
Now let's get the details for our support ticket. For that you will need to provide the ticket id just generated in the previous query to `create_ticket`

In [ ]:
%%time
response = agents.invoke(
    "Can I get details on my ticket? My store id is 1 get my ticket id from our previous conversation {}".format(ticket), 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(response)

### Testing get open support tickets
Let's also check if our agent can get the details for more than one open case. To do so we will first create a new support case

In [ ]:
%%time
response = agents.invoke(
    "I need help with inventory rotation procedures for Standard Collection items. My store id is 1", 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(response)

Next we will request the overview of all open tickets

In [ ]:
%%time
response = agents.invoke(
    "Can I get all tickets that I have? My store id is 1", 
    inventory_agent[0], enable_trace=True
)
print("====================")
print(response)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
inventory_agent_alias_id, inventory_agent_alias_arn = agents.create_agent_alias(
    inventory_agent[0], 'v1'
)
inventory_agent_id = inventory_agent[0]

Store environment variables to be used on next notebooks.

In [ ]:
inventory_agent_arn = agents.get_agent_arn_by_name(inventory_agent_name)
inventory_kb = knowledge_base_name
inventory_dynamodb = dynamodb_table

%store inventory_agent_arn
%store inventory_agent_alias_arn
%store inventory_agent_alias_id
%store inventory_lambda_name
%store inventory_agent_name
%store inventory_agent_id
%store inventory_kb
%store inventory_dynamodb

In [ ]:
inventory_agent_arn, inventory_agent_alias_arn, inventory_agent_alias_id

## Next Steps
Congratulations! We've now created an inventory management agent. Next we will create our store operations manager agent